In [41]:
! pip install opencv-python


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [42]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)  # Flip to avoid mirror effect
    cv2.rectangle(frame, (100, 100), (300, 300), (0, 255, 0), 2)
    roi = frame[100:300, 100:300]

    cv2.imshow("Frame", frame)

    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# Define skin color range in HSV
lower_skin = np.array([0, 20, 70], dtype=np.uint8)
upper_skin = np.array([20, 255, 255], dtype=np.uint8)

# Threshold the HSV image to get skin color
mask = cv2.inRange(hsv, lower_skin, upper_skin)

# Clean up using morphological operations
mask = cv2.dilate(mask, np.ones((3, 3), np.uint8), iterations=4)
mask = cv2.GaussianBlur(mask, (5, 5), 100)

cv2.imshow("Mask", mask)

KeyboardInterrupt: 

In [ ]:
import cv2
import numpy as np

# Access webcam
cap = cv2.VideoCapture(0)  # 0 = default camera

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)  # Flip the frame for mirror effect

    # Define Region of Interest (ROI) for hand
    roi = frame[100:300, 100:300]  # x1:y1 to x2:y2

    # Convert ROI to HSV color space
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    # Define HSV range for skin color
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)

    # Create a binary mask for skin color
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Improve mask using morphological operations
    mask = cv2.dilate(mask, np.ones((3, 3), np.uint8), iterations=4)
    mask = cv2.GaussianBlur(mask, (5, 5), 100)

    # Draw rectangle around ROI in the original frame
    cv2.rectangle(frame, (100, 100), (300, 300), (0, 255, 0), 2)

    # Display original frame and mask side by side
    cv2.imshow("Frame", frame)
    cv2.imshow("Skin Mask", mask)

    # Break loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release camera and close windows
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:
import cv2
import numpy as np

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # Flip for mirror effect

    # Define Region of Interest (ROI) for hand
    roi = frame[100:300, 100:300]
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    # Define HSV range for skin detection
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)

    # Mask skin color region
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Improve the mask using morphological operations
    mask = cv2.dilate(mask, np.ones((3, 3), np.uint8), iterations=4)
    mask = cv2.GaussianBlur(mask, (5, 5), 100)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        max_contour = max(contours, key=cv2.contourArea)

        # Draw the hand contour
        cv2.drawContours(roi, [max_contour], -1, (0, 255, 0), 2)

        # Convex Hull
        hull = cv2.convexHull(max_contour)
        cv2.drawContours(roi, [hull], -1, (0, 0, 255), 2)

        # Convexity Defects (to count fingers)
        hull_indices = cv2.convexHull(max_contour, returnPoints=False)

        if hull_indices is not None and len(hull_indices) > 3:
            defects = cv2.convexityDefects(max_contour, hull_indices)

            if defects is not None:
                count = 0
                for i in range(defects.shape[0]):
                    s, e, f, d = defects[i, 0]
                    start = tuple(max_contour[s][0])
                    end = tuple(max_contour[e][0])
                    far = tuple(max_contour[f][0])

                    # Apply cosine rule to find angle
                    a = np.linalg.norm(np.array(end) - np.array(start))
                    b = np.linalg.norm(np.array(far) - np.array(start))
                    c = np.linalg.norm(np.array(end) - np.array(far))
                    angle = np.arccos((b**2 + c**2 - a**2) / (2 * b * c))

                    # Count as finger if angle < 90 degrees
                    if angle <= np.pi / 2:
                        count += 1
                        cv2.circle(roi, far, 5, [255, 0, 0], -1)

                # Display number of fingers
                cv2.putText(frame, f"Fingers: {count + 1}", (50, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Draw ROI box
    cv2.rectangle(frame, (100, 100), (300, 300), (0, 255, 0), 2)

    # Show output
    cv2.imshow("Skin Mask", mask)
    cv2.imshow("Gesture", frame)

    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [44]:
import cv2
import numpy as np

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)

    # Define Region of Interest (ROI)
    roi = frame[100:300, 100:300]
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    # Define HSV range for skin color
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)

    # Create skin mask
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    mask = cv2.dilate(mask, np.ones((3, 3), np.uint8), iterations=4)
    mask = cv2.GaussianBlur(mask, (5, 5), 100)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    gesture_text = ""  # Default gesture

    if contours:
        max_contour = max(contours, key=cv2.contourArea)
        cv2.drawContours(roi, [max_contour], -1, (0, 255, 0), 2)

        hull = cv2.convexHull(max_contour)
        cv2.drawContours(roi, [hull], -1, (0, 0, 255), 2)

        hull_indices = cv2.convexHull(max_contour, returnPoints=False)
        if hull_indices is not None and len(hull_indices) > 3:
            defects = cv2.convexityDefects(max_contour, hull_indices)

            if defects is not None:
                count = 0
                for i in range(defects.shape[0]):
                    s, e, f, d = defects[i, 0]
                    start = tuple(max_contour[s][0])
                    end = tuple(max_contour[e][0])
                    far = tuple(max_contour[f][0])

                    a = np.linalg.norm(np.array(end) - np.array(start))
                    b = np.linalg.norm(np.array(far) - np.array(start))
                    c = np.linalg.norm(np.array(end) - np.array(far))
                    angle = np.arccos((b*2 + c*2 - a*2) / (2 * b * c))

                    if angle <= np.pi / 2:
                        count += 1
                        cv2.circle(roi, far, 5, [255, 0, 0], -1)

                # Map count to text
                if count == 0:
                    gesture_text = "ONE"
                elif count == 1:
                    gesture_text = "TWO"
                elif count == 2:
                    gesture_text = "THREE"
                elif count == 3:
                    gesture_text = "FOUR"
                elif count == 4:
                    gesture_text = "FIVE"
                else:
                    gesture_text = "UNKNOWN"

                # Display gesture name
                cv2.putText(frame, f"Gesture: {gesture_text}", (50, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Draw rectangle around ROI
    cv2.rectangle(frame, (100, 100), (300, 300), (0, 255, 0), 2)

    # Show the output
    cv2.imshow("Skin Mask", mask)
    cv2.imshow("Gesture", frame)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 